In [58]:
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix
from scipy.sparse import hstack as sp_hstack
from scipy.sparse import diags as spdiags
from scipy.sparse import eye as speye

from tqdm import tqdm
from polara.preprocessing.dataframes import matrix_from_observations

from lfm import build_lfm_model, encode_virus_features
from evaluation import topn_recommendations

# Setting up Kaggle

For your convenience, we've setup Kaggle leaderboard for this home assignment. This will help you to immediately get an understanding of whether you're on the right way. If you submission gets you 0 score, something is wrong with your implementation.

To submit your solutions automatically, you need to 
1. Register at the competition page. Use [this link](https://www.kaggle.com/t/4c8fd62231ce4308806b642ac6002c07) for registering.
2. Setup Kaggle API client via `pip install kaggle` into your environment.
3. Store your Kaggle API credentials locally. See [this instruction](https://github.com/Kaggle/kaggle-api#api-credentials) for details.

Before uploading the solution, convert it into the proper format and save it locally, using the following function:

In [59]:
def save_submission(recs, test_user_index, item_index, name='kaggle_submission.csv'):
    """
    Saves the recommendations for test users in a CSV file in a proper format,
    ready for upload as a Kaggle submission.
    It also converts internal indices into their original external representation.

    Parameters
    ----------
    recs : numpy.ndarray
        An array of shape (num_test_users, num_recommendations) containing the
        indices of the recommended items.
    test_user_index : pd.Index
        Pandas Index containing the external test user indices.
    item_index : pd.Index
        Pandas Index containing the index mapping for the item indices.
    name : str, optional
        Name of the output file. Defaults to 'kaggle_submission.csv'.

    Returns:
        None 
    """
    submission = pd.Series(
        data = list(item_index.take(recs).values),
        index = test_user_index,
        name = 'compound_id'
    ).apply(lambda x: ' '.join([str(i) for i in x])).reset_index()
    submission.to_csv(name, index=False)

After the solution is saved, you can upload it using the command in the following format:

```
!kaggle competitions submit -c {competition_name} -f {submission_file} -m 'my submision name'
```

The competition name is provided below. Do not change it.

In [60]:
competition_name = 'anti-viral-drug-discovery-skoltech-recsys-course'

**NOTE #1:** Kaggle only allows up to 20 submissions per day. Try to use the submissions budget wisely.  

**NOTE #2:** If you find it more convenient, you can still submit solutions manually without using the code provided above.

# Preparing Data

You will work with two sources of data:  
- activation data (`virus_chem_activation.csv`)
  - describes if there is a positive (+1) or negative (-1) response from a virus to a chemical compound
    - positive activation means that the compound is likely to be effective against the virus
    - negative activation means the the virus is chemically resistant to the compound
- virus family data (`virus_features.csv`)
  - describes which viral family class each virus belongs to


Your target viruses are listed below by their `virus_id`:

In [61]:
cold_start_viruses = pd.Index([599, 757, 1076, 1200, 1421, 1970, 2170, 2724], name='virus_id')


Your global task:
 - for each target virus, identify top-5 compounds that might have positive activation on them.
 
 Note that unlike typical scenario, in this case, the target viruses must maximally dislike the recommendations. :)

## Data for binary classification? (5 pts)

Would it be possible to simply formulate the task in terms of a binary classification with  `[+1.0, -1.0]` labels ? Describe the pros and cons of such an approach? Is it applicable in the current task?

**Hint:** How many types of "feedback" do we actually have? Is it just two?

#your answer here  
...

## Load data

In [62]:
data = pd.read_csv('virus_chem_activation.csv')
data.tail()

,compound_id,virus_id,activated
40472,516059,3174,-1.0
40473,516063,3174,1.0
40474,573139,3174,-1.0
40475,615133,3174,-1.0
40476,1303637,3174,1.0


In [63]:
data.groupby('activated').nunique()

,compound_id,virus_id
activated,,
-1.0,4344,88
1.0,3791,100


In [64]:
virus_families = pd.read_csv('virus_features.csv')
virus_families.head()

,virus_id,family
0,103,1814
1,480,1882
2,481,1882
3,483,1882
4,502,1884


## Check data consistency (total 5 pts)

### cold start setting  (2 pts)

Recall that scientific labs were not able to identify compounds that would be effective against the target viruses.  

Veryfy that there is `no positive activation` data for these viruses in the provided dataset:

In [65]:
# your code here:
data.set_index('virus_id').loc[cold_start_viruses].activated.sum() + len(data.set_index('virus_id').loc[cold_start_viruses].activated.values)

0.0

### lack of features (3 pts)

Recall that despite the joint attempts of many scientits, only general families of the new viruses were identified. These families may not be necessarily associated with compounds that give positive activations against the viruses of the family. 

Verify this by calculating the proportion of families of target viruses among all families associated with positive activation against at least one compound.

In [66]:
# your code here
num_active_families = 0
for fam in virus_families.family.unique():
    activated = False
    for virus in virus_families.groupby('family').get_group(fam).values[:, 0]:
        if virus in data.virus_id.unique():
            if 1.0 in data[['virus_id', 'activated']].groupby('virus_id').get_group(virus).values[:, 1]:
                activated = True
    if activated:
        num_active_families += 1
virus_families.set_index('virus_id').loc[cold_start_viruses].family.nunique() / num_active_families

0.18181818181818182

## Feature pre-processing (total 5 pts)

- Your task now is to design a proper hybrid recommender for the cold start setting considering all the features that you were provided with. The solution will be based on the [`LightFM`](https://github.com/lyst/lightfm/) approach.

- As you have already verified, the `virus family` feature is not sufficient for the task, as it is not fully covered by the positive activation data. Some additional information must be used.

Before proceeding, try to pause here and think what esle can be used as an additional feature?  
- **Hint #1**: Recall NSVD or SVD++ models. What were the design matrices composed of in their case?
- **Hint #2**: How many types of interactions are there in the activation data?

### resistance feature

Let's denote the ability of viruses to remain intact under action of chemical comounds as `resistance`.

This type of information corresponds to the `activated == -1.0` values in the provided activation dataset. You have probably already guessed that this is exactly the type of feature that you can additionally use in construction of a hybrid recommender.

- Prepare the `resistance feature` dataset by taking out the corresponding entries from the activation dataset.
- Store the data in the `resistance` dataframe.
- Put the positive activation data into a separate `train_data` dataframe. This dataframe will be used for training.

#### extracting additional feature data (1 pts)

In [67]:
data.groupby('activated').get_group(-1.0)[['compound_id', 'virus_id']]

,compound_id,virus_id
0,24630,103
1,451356,103
2,525524,103
3,525528,103
4,541277,103
...,...,...
40446,1349220,2989
40471,468192,3174
40472,516059,3174
40474,573139,3174


In [68]:
# your code here:
resistance =  data.groupby('activated').get_group(-1.0)[['compound_id', 'virus_id']]

In [69]:
resistance.nunique()

compound_id    4344
virus_id         88
dtype: int64

#### extracting training data (1 pts)

In [70]:
# your code
train_data = data.groupby('activated').get_group(1.0)[['virus_id', 'compound_id']]

In [71]:
train_data.nunique()

virus_id        100
compound_id    3791
dtype: int64

It's a good idea to immediately transform the training data into a convenient sparse format and to build the corresponding index of viruses and compounds. You did it several times throughout the course already. Below is an example of how it can be achieved with the `matrix_from_observations` function form `polara`. No additional code is required here.

In [72]:
train_matrix, virus_index, compound_index = matrix_from_observations(
    train_data, userid='virus_id', itemid='compound_id'
)
train_matrix

<100x3791 sparse matrix of type '<class 'numpy.int64'>'
	with 20105 stored elements in Compressed Sparse Row format>

### transforming features into proper format (3 pts)

According to the LightFM documentation, features must be represented in the format of `sparse matrices` with
- rows corresponding to an entity the feature describes and
- columns corresponding to the feature values.

You are provided with the convenience function named `encode_virus_features` which takes as an input:
- a virus feature dataframe,
- an array of viruses (in the form of `pandas Index`) which features are to be encoded
- the name of the columns corresponding to the selected feature type

and returns
- a sparse binary feature matrix (one-hot encoding of features) and
- the corresponding feature index that maps columns of the matrix back into the original features of viruses from the training.

The feature index will be later used for preprocessing the features of the target viruses.

#### resistance into sparse format

In [73]:
# your code here:
virus_strength_feature_matrix, virus_strength_feature_idx = encode_virus_features(
     data=resistance, virus_index=virus_index, feature_field='compound_id'
)
virus_strength_feature_matrix

<100x4313 sparse matrix of type '<class 'numpy.float64'>'
	with 19744 stored elements in Compressed Sparse Row format>

In [74]:
assert virus_strength_feature_matrix.shape[0] == len(virus_index)

#### virus family into sparse format

Do the same transformation for the `virus family` feature below:

In [75]:
# your code here:
virus_family_feature_matrix, virus_family_feature_idx = encode_virus_features(
    data=virus_families, virus_index=virus_index, feature_field='family'
)
virus_family_feature_matrix

<100x44 sparse matrix of type '<class 'numpy.float64'>'
	with 100 stored elements in Compressed Sparse Row format>

In [76]:
assert virus_family_feature_matrix.shape[0] == len(virus_index)

## Transform target viruses' features into sparse format (total 2 pts)

You also need to perform the corresponding transformation of features for the target viruses that are not part of the train by construction.

- Recall that not all target viruses have their `family` feature represented in the train data. The same holds for the `resistance` feature as well.
- The `encode_virus_features` takes care of that by filtering out the non-consistent features that cannot be a part of the training.
- To make it work you just need to provide the proper index of the features obtained at the earlier steps of the feature transformation:
  - the index must be provided via an optional argument `feature_index` (must be of `pandas Index` type),
  - mind that all features that are not covered by the provided feature index must be discarded (controlled by the `drop_invalid` argument, `True` by default)

### resistance (1 pts)

In [77]:
# fill in the missing parts of the code:
virus_cold_start_strength_feature_matrix, virus_cold_start_strength_feature_idx = encode_virus_features(
    data = resistance,
    virus_index = cold_start_viruses,
    feature_field = 'compound_id',
    feature_index = virus_strength_feature_idx, # provide the proper feature index
    drop_invalid = True  # filter out features inconsistent with train data
)
virus_cold_start_strength_feature_matrix

<8x4313 sparse matrix of type '<class 'numpy.float64'>'
	with 77 stored elements in Compressed Sparse Row format>

### family (1 pts)

In [78]:
# fill in the missing parts of the code:
virus_cold_start_family_feature_matrix, virus_cold_start_family_feature_idx = encode_virus_features(
    data = virus_families,
    virus_index = cold_start_viruses,
    feature_field = 'family',
    feature_index = virus_family_feature_idx, # provide the proper feature index
    drop_invalid = True  # filter out features inconsistent with train data
)
virus_cold_start_family_feature_matrix

<8x44 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

# Building the hybrid recommender model

Your task is to gradually improve the model following the instructions provided below.

## Initial setup for LightFM

### data description

In [79]:
data_description = dict(
    users = 'virus_id',
    items = 'compound_id',
    n_users = len(virus_index),
    n_items = len(compound_index),
    # user_features = ... skip for now, will be defined later
    item_features = None # compound features are not present in the dataset
)
data_description

{'users': 'virus_id',
 'items': 'compound_id',
 'n_users': 100,
 'n_items': 3791,
 'item_features': None}

You will use a separate data description dictionary for generating predictions. This dictionary will provide the necessary information about the target viruses.

In [80]:
data_description_cold_start = dict(
    users = 'virus_id',
    items = 'compound_id',
    n_cold_users = len(cold_start_viruses),
    n_items = len(compound_index),
    # user_features = ... skip for now, will be defined later
    item_features = None # compound features are not present in the dataset
)
data_description_cold_start

{'users': 'virus_id',
 'items': 'compound_id',
 'n_cold_users': 8,
 'n_items': 3791,
 'item_features': None}

### hyper-parameters

In [81]:
lfm_config = dict(
    no_components = 60,
    loss = 'warp',
    max_sampled = 1,
    max_epochs = 60,
    learning_schedule = 'adagrad',
    user_alpha = 1e-3,
    item_alpha = 1e-3,
    random_state = 7032023
)

### evaluation

In [82]:
topn = 5

## The simplest case - using the "family" feature (total 10 pts)

### training

In [83]:
data_description['user_features'] = virus_family_feature_matrix

In [84]:
lfm_model = build_lfm_model(
    lfm_config,
    train_matrix,
    data_description,
    iterator = tqdm
)

100%|██████████| 60/60 [00:00<00:00, 145.46it/s]


### generating predictions

#### Implement scoring function (10 pts)

In [85]:
# your code here
def lightfm_scoring_base(model, data, data_description):
    """
    A standard scoring function adopted for use with LightFM in the user cold-start settings.
    """
    dtype = 'i4'
    
    all_items = np.arange(data_description['n_items'], dtype=dtype)
    test_users = np.arange(data_description['n_cold_users'], dtype=dtype)
    user_index, item_index = np.meshgrid(test_users, all_items, copy=False)
    lfm_scores = model.predict(
        user_ids = user_index.ravel(),
        item_ids = item_index.ravel(),
        item_features = None,
        user_features = data_description['user_features']
    )
    scores = lfm_scores.reshape(len(test_users), len(all_items), order='F')    
    
    return scores

Use `data_description_cold_start` dictionary to initialize the required variables.

In [86]:
data_description_cold_start['user_features'] = virus_cold_start_family_feature_matrix

In [87]:
lfm_scores = lightfm_scoring_base(lfm_model, None, data_description_cold_start)
lfm_recs = topn_recommendations(lfm_scores, topn=topn)

#### Evaluate the result using Kaggle

In [88]:
# kaggle_submission_name = 'base_model.csv'

# save_submission(
#     lfm_recs, cold_start_viruses, compound_index, name=kaggle_submission_name
# )

In [89]:
# !kaggle competitions submit -c {competition_name} -f {kaggle_submission_name} -m 'base model v.1'

Please also report the obtained score on the Kaggle leaderboard here.

The score is 0.06896

### handling biases (total 13 pts)

It is perhaps reasonable to assume that most popular and effective compounds are known to scientists and most likely have been tried out in the first place with the new viruses. As there's still no positive activation data, it means that the predictive model must avoid favoring popular compounds in recommendations.

Recall that, in the `LightFM` model, the major popularity biases are captured in the linear bias terms. Also note that:
- when measured across different items, the variance of the bias prediction term $\operatorname{Var}_y\left[b^\top y\right]$ can be higher than the variance of the factorization term $\operatorname{Var}_y\left[x^\top PQ^\top y\right]$
  - if this is the case, the difference in predicted scores related to the factorization term may become negligible and won't affect ranking of the items;
  - in other words, the recommendations of your model may promote items with the highest bias values and provide little to no personalization.
- Unfortunately, `LightFM` neither lets you to disable biases nor does it provide a separate regularisation coefficient for linear biases (so that their influence could be decreased).
- Hence, you will use a simple heuristic - just set biases to 0 when generating prediction scores


**The task**:
- implement scoring function that allows disabling linear bias terms for items. (10 pts)
  - implement a new variant of the scoring function that takes an additional **boolean** argument `item_bias` as input
    - if `item_bias` is `False`, the function must temporarily set the corresponding bias terms to 0 when computng prediction scores
  - you can access `LightFM`'s biases array via `model.item_biases` variable (see [documentation](https://making.lyst.com/lightfm/docs/lightfm.html))
    - make sure to store the current values of biases in a temporary array using the `numpy`'s `.copy()` function
    - set all item bias weights to zero via an inplace operation `*= 0`
  - don't forget to restore biases once the the prediction scores are generated;:
    - use the `model.item_biases[:] = ...` syntax to make sure you're not creating a new varibale

You are only required to handle biases related to compounds.  
- Explain, why the linear bias term related to viruses do not play any role in the task? (3 pts)

In [1]:
# your code here
def lightfm_scoring(model, data, data_description, item_bias=True):
    dtype = 'i4'
    
    all_items = np.arange(data_description['n_items'], dtype=dtype)
    test_users = np.arange(data_description['n_cold_users'], dtype=dtype)
    user_index, item_index = np.meshgrid(test_users, all_items, copy=False)
    
    
    biases = model.item_biases.copy()
    if not item_bias:
        model.item_biases[:] = np.zeros_like(model.item_biases)
        
    lfm_scores = model.predict(
            user_ids = user_index.ravel(),
            item_ids = item_index.ravel(),
            item_features = None,
            user_features = data_description['user_features']
        )
    
    model.item_biases[:] = biases
        
    scores = lfm_scores.reshape(len(test_users), len(all_items), order='F') 
        
    return scores

#### regenerating scores with disabled biases

In [91]:
lfm_scores = lightfm_scoring(lfm_model, None, data_description_cold_start, item_bias=False)
lfm_recs = topn_recommendations(lfm_scores, topn=topn)

#### evaluate the result using Kaggle

In [92]:
# kaggle_submission_name = 'debiased_model.csv'

# save_submission(
#     lfm_recs, cold_start_viruses, compound_index, name=kaggle_submission_name
# )

In [93]:
# !kaggle competitions submit -c {competition_name} -f {kaggle_submission_name} -m 'debiased model v.1'

Please also report the obtained score on the Kaggle leaderboard here.

Was the hypothesis about the influence of bias terms confirmed?

The score increased to 0.10344.

## Improved model (total 10 pts)

Note that so far you have not implemented a truly hybrid model:
- By default, `LightFM` utilizes information about user and item `id`'s only when no feature matrices are provided (unless you use the `lightfm.data.Dataset` class, see [documentation](https://making.lyst.com/lightfm/docs/lightfm.html)).
- In the current setup, with explicitly provided `virus family` feature matrix, `LightFM` does not automatically extend the feature space with the `id`'s of the viruses.

Hence, the collaborative information is not yet utilized in the best possible way:
- a virus is simply represented as a bag of features.
- Recall that several viruses can belong to the same family.
  - Hence, the **expressiveness of the current virus representation is limited**.

The task: (5 pts)
- In order to resolve this issue, you need to manually extend representation of viruses by adding one-hot encoding of their `id`'s into the feature matrix.
- You can use functions called `eye` and `hstack` from `scipy.sparse`.
  - These functions are already imported in this notebook and accessible via the names `speye` and `sp_hstack`.

In [94]:
data_description

{'users': 'virus_id',
 'items': 'compound_id',
 'n_users': 100,
 'n_items': 3791,
 'item_features': None,
 'user_features': <100x44 sparse matrix of type '<class 'numpy.float64'>'
 	with 100 stored elements in Compressed Sparse Row format>}

In [95]:
# your code here

data_description['user_features'] = sp_hstack(
    [speye(data_description['user_features'].shape[0]), 
     virus_family_feature_matrix]
)
data_description

{'users': 'virus_id',
 'items': 'compound_id',
 'n_users': 100,
 'n_items': 3791,
 'item_features': None,
 'user_features': <100x144 sparse matrix of type '<class 'numpy.float64'>'
 	with 200 stored elements in COOrdinate format>}

### retrain the extended model

In [96]:
lfm_model = build_lfm_model(
    lfm_config,
    train_matrix,
    data_description,
    iterator = tqdm
)

100%|██████████| 60/60 [00:00<00:00, 145.36it/s]


### generate recommendations (5 pts)

You also need to update the representation for the target viruses.

- Note that there is no `id` feature for the target viruses in the model because they are not part of the training data.
- Hence, you simply need to add an empty block of the proper shape to conform with the extended feature space.
    - Use `csr_matrix` constructor to generate an empty matrix of conforming size

In [97]:
data_description_cold_start

{'users': 'virus_id',
 'items': 'compound_id',
 'n_cold_users': 8,
 'n_items': 3791,
 'item_features': None,
 'user_features': <8x44 sparse matrix of type '<class 'numpy.float64'>'
 	with 4 stored elements in Compressed Sparse Row format>}

In [98]:
data_description

{'users': 'virus_id',
 'items': 'compound_id',
 'n_users': 100,
 'n_items': 3791,
 'item_features': None,
 'user_features': <100x144 sparse matrix of type '<class 'numpy.float64'>'
 	with 200 stored elements in COOrdinate format>}

In [99]:
# your code here

data_description_cold_start['user_features'] = sp_hstack(
    [csr_matrix((data_description_cold_start['n_cold_users'], data_description['n_users'])),
        virus_cold_start_family_feature_matrix]
)
data_description_cold_start

{'users': 'virus_id',
 'items': 'compound_id',
 'n_cold_users': 8,
 'n_items': 3791,
 'item_features': None,
 'user_features': <8x144 sparse matrix of type '<class 'numpy.float64'>'
 	with 4 stored elements in Compressed Sparse Row format>}

In [100]:
lfm_scores = lightfm_scoring(lfm_model, None, data_description_cold_start, item_bias=False)
lfm_recs = topn_recommendations(lfm_scores, topn=topn)

### evaluate the result using Kaggle

In [101]:
# kaggle_submission_name = 'improved_model.csv'

# save_submission(
#     lfm_recs, cold_start_viruses, compound_index, name=kaggle_submission_name
# )

In [102]:
# !kaggle competitions submit -c {competition_name} -f {kaggle_submission_name} -m 'improved model v.1'

Please report the obtained score on the Kaggle leaderboard here.

- Were you able to improve the metric?
- How does enabling/disabling item biases affect the result?

If i disable biases, it has the same score as previous submission (0.13793 with item_bias = False and 0.10344 with item_bias = True)

## Including the virus resistance feature (total 15 pts)

Following the same steps as before, construct an extended representation of virus features that utilizes of three types of information:
- collaborative
- virus family features
- virus resistance against compounds


Note that each virus may have been testsed against several different compounds with negative activation result.
- Hence, the two types of features -- `family` and `resistance` -- will have an unequal contribution into the feature representation.
- The `resistance` feature will dominate for viruses with a lot of experimental data.

Your task now is not only to extend the feature space but also to properly normalize the `resistance` feature so that its contribution becomes comparable to that of the `family` feature.

### extending feature space with the normalized resistance feature

You need to perform two steps:
- Implement a simple feature normalization function that will normalize each row in a feature matrix by the number of non-zero elements in this row.
  - You can use `diags` function from `scipy.sparse` for weights. It is already imported in the notebook under the name `spdiags`.
- Apply the function to the `resistance` feature matrix before extending the feature space. (5 pts)

In [103]:
def normalize_features(feature_matrix):
    '''
    Normalize the input sparse binary feature matrix for every row.

    Parameters
    ----------
    feature_matrix : scipy.sparse.csr_matrix
        The input feature matrix, where each row is a binary feature vector.

    Returns
    -------
    scipy.sparse.csr_matrix
        The normalized feature matrix, where each row is a probability distribution 
        over the features.

    Examples
    --------
    For a feature matrix (take A as a sparse matrix) A= [[1, 0, 1], [0, 0, 1], [1, 1, 1]], 
    the normalized feature matrix is [[0.5, 0, 0.5], [0, 0, 1], [0.333, 0.333, 0.333]].

    Take into account that this function takes and returns sparse matrices. 
    Dense matrices are used here for the sake of example.
    >>> A = np.array([[1, 0, 1], [0, 0, 1], [1, 1, 1]])
    >>> normalize_features(spdiags(A).tocsr())
    array([[0.5  , 0.   , 0.5  ],
           [0.   , 0.   , 1.   ],
           [0.333, 0.333, 0.333]])

    Notes
    -----
    The normalization is performed by dividing each row by the sum of its non-zero 
    elements. If a row has all zeros, it is left unchanged.
    '''
    # your code here
    sums = np.array(feature_matrix.sum(axis=1))
    sums[sums == 0] = 1
    return feature_matrix / sums
    

Use the `normalize_features` function that you implemented to get the extended feature representation that includes all three type of information: (10 pts)
- colaborative
- feature family
- resistance

In [104]:
# your code here


data_description['user_features'] = sp_hstack(
    [speye(data_description['user_features'].shape[0]), 
     virus_family_feature_matrix,
     normalize_features(virus_strength_feature_matrix)]
)

data_description_cold_start['user_features'] = sp_hstack(
    [csr_matrix((data_description_cold_start['n_cold_users'], data_description['n_users'])),
        virus_cold_start_family_feature_matrix,
        normalize_features(virus_cold_start_strength_feature_matrix)]
)

### generate recommendations

In [105]:
lfm_model = build_lfm_model(
    lfm_config,
    train_matrix,
    data_description,
    iterator = tqdm
)

100%|██████████| 60/60 [00:12<00:00,  4.95it/s]


In [106]:
lfm_scores = lightfm_scoring(lfm_model, None, data_description_cold_start, item_bias=True)
lfm_recs = topn_recommendations(lfm_scores, topn=topn)

### evaluate the result using Kaggle

In [107]:
# kaggle_submission_name = 'all_features_model.csv'

# save_submission(
#     lfm_recs, cold_start_viruses, compound_index, name=kaggle_submission_name
# )

In [108]:
# !kaggle competitions submit -c {competition_name} -f {kaggle_submission_name} -m 'all features model v.1'

Please report the obtained score on the Kaggle leaderboard here.

- Were you able to improve the metric?

Yes, it has improved. The score now is 0.17241

Before proceeding, try to explain why you obtained this result after modifying the feature space.

## Relative feature weighting (total 10 pts)

Note that once all three types of features are added, their relative contribution uniformly decreases.
- In the beginning, you had collaborative and `family` features, so their relative contribution was one half.
- With extended representation the contribution is one third for each type.

Hypothesis:
- Reducing the relative contribution of collaborative part is likely to be the cause of the quality drop.

The task:
- Implement additional reweighting so that the contribution of collaborative part is one half again.
- The distribution of relative contribution weights between the `family` and the `resistance` feature must be controlled via an additional scalar hyper-parameter $\alpha \in [0, 1]$, i.e.,
  - all the `family` features must be globally multiplied by $\alpha$,
  - all the `resistance` features must be globally multiplied by $1-\alpha$.
    - That way, the scenario with only collaborative and `family` features is covered by setting $\alpha=1$.
    - Likewise, excluding `family` feature will correspond to $\alpha=0$.

In [139]:
family_feature_weight = 0.45 # the alpha hyper-parameter

Define the reweightend feature space using the `family_feature_weight` value: (10 pts)

In [140]:
# your code here
data_description['user_features'] = sp_hstack(
    [speye(data_description['user_features'].shape[0]), 
     family_feature_weight * virus_family_feature_matrix,
     (1.0 - family_feature_weight) * normalize_features(virus_strength_feature_matrix)]
)

data_description_cold_start['user_features'] = sp_hstack(
    [csr_matrix((data_description_cold_start['n_cold_users'], data_description['n_users'])),
        family_feature_weight * virus_cold_start_family_feature_matrix,
        (1.0 - family_feature_weight) * normalize_features(virus_cold_start_strength_feature_matrix)]
)

### generate recommendations

In [141]:
lfm_model = build_lfm_model(
    lfm_config,
    train_matrix,
    data_description,
    iterator = tqdm
)

100%|██████████| 60/60 [00:12<00:00,  4.77it/s]


In [142]:
lfm_scores = lightfm_scoring(lfm_model, None, data_description_cold_start, item_bias=False)
lfm_recs = topn_recommendations(lfm_scores, topn=topn)

### evaluate the result using Kaggle

In [143]:
kaggle_submission_name = 'reweighted_features_model.csv'

save_submission(
    lfm_recs, cold_start_viruses, compound_index, name=kaggle_submission_name
)

In [144]:
!kaggle competitions submit -c {competition_name} -f {kaggle_submission_name} -m 'reweighted features model 0.45'

100%|████████████████████████████████████████████| 349/349 [00:00<00:00, 428B/s]
Successfully submitted to Anti-viral drug discovery - Skoltech RecSys course

Please report the obtained score on the Kaggle leaderboard here. (5 pts)

- Were you able to improve the metric?
- Was the hypothesis confirmed?

I tried $\alpha=0.4$, and got 0.20689, it improved. 0.5, 0.8 gave the same result, also tried 1.0 (the same as result above in normalization) and 0.0 (got 3%, worst of all). 0.35, 0.45, 0.2 got around 10%. I'd say hypothesis doesn't work, since the score was greater than without weighting just for one value of parameter, but decreased for a bunch of other values of parameters.